In [1]:
from data.Project_2_People_Counter.sort import *

In [4]:
import cv2
from ultralytics import YOLO
import cvzone
import math
import numpy as np

cap = cv2.VideoCapture("data/Videos/people.mp4")
cap.set(3, 1280)
cap.set(4, 720)

model = YOLO("Yolo-Weights/yolov8n.pt")

classNames = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
               "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
               "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
               "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
               "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
               "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop",
               "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
               "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
               "toothbrush"]

mask = cv2.imread("data/Project_2_People_Counter/mask.png")
tracker = Sort(max_age=20, min_hits=2,iou_threshold=0.3)
limitsUp = [103, 161, 296, 161]
limitsDown = [527, 489, 735, 489]
totalCountUp = []
totalCountDown = []

while 1:
    ret, img = cap.read()
    if not ret:
        break

    imgRegion = cv2.bitwise_and(img, mask)

    imgGraphics = cv2.imread("data/Project_2_People_Counter/graphics.png", cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (730, 260))

    results = model(imgRegion, stream= True)
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
            w, h = x2-x1, y2-y1

            conf = math.ceil((box.conf[0] *100))/100
            # cvzone.putTextRect(img, f"{conf}", (x1,y1))

            cls  = int(box.cls[0])

            currenClass = classNames[cls]
            if (currenClass == "person") and (conf > 0.3):
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt= 5)
                currenArray = np.array([x1, y1, x2, y2, conf])
                # np.vstack là một hàm trong NumPy được sử dụng để nối các mảng theo chiều dọc.
                # detections là một mảng đã tồn tại trước đó.
                # currentArray là một mảng khác có cùng số lượng cột với detections.
                detections = np.vstack((detections, currenArray))
    
    resultsTracker = tracker.update(detections)
    cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 0, 255), 5) 
    cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 0, 255), 5) 

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2-x1, y2-y1
        cvzone.cornerRect(img, (x1, y1, w, h), l = 9, rt= 2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f'{int(id)}', (max(0,x1), max(35,y1)), 
                               scale= 2, thickness=3, offset=10)

        cx, cy = x1 + w//2, y1 +h//2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        if (limitsUp[0] < cx < limitsUp[2]) and (limitsUp[1] - 15 < cy < limitsUp[1] + 15):
            if totalCountUp.count(id) == 0:
                totalCountUp.append(id)
                cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 255, 0), 5) 
        
        if (limitsDown[0] < cx < limitsDown[2]) and (limitsDown[1] - 15 < cy < limitsDown[1] + 15):
            if totalCountDown.count(id) == 0:
                totalCountDown.append(id)
                cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 255, 0), 5)
    
    cv2.putText(img,str(len(totalCountUp)),(929,345),cv2.FONT_HERSHEY_PLAIN,5,(139,195,75),7)
    cv2.putText(img,str(len(totalCountDown)),(1191,345),cv2.FONT_HERSHEY_PLAIN,5,(50,50,230),7)
    cv2.imshow("count people", img)
    if cv2.waitKey(1) & 0xFF == 27:
        break

    
cap.release()
cv2.destroyAllWindows()


0: 384x640 3 persons, 123.7ms
Speed: 12.0ms preprocess, 123.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 141.9ms
Speed: 0.0ms preprocess, 141.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 125.1ms
Speed: 0.0ms preprocess, 125.1ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 117.4ms
Speed: 0.0ms preprocess, 117.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 122.4ms
Speed: 0.0ms preprocess, 122.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 121.4ms
Speed: 0.0ms preprocess, 121.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 120.4ms
Speed: 2.1ms preprocess, 120.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 111.0ms
Speed: 6.3ms preprocess, 111.0ms inference, 0.0ms postprocess per